In [1]:
import torch
import copy
import syft as sy
from torch import nn
from torch import optim
hook = sy.TorchHook(torch)

# 1. Make fake worker nodes

In [2]:
alex = sy.VirtualWorker(hook, id='alex')
toby = sy.VirtualWorker(hook, id='toby')
jake = sy.VirtualWorker(hook, id='jake')
matt = sy.VirtualWorker(hook, id='matt')
shreyas = sy.VirtualWorker(hook, id='shreyas')

# secure worker node built as well
secure_machine = sy.VirtualWorker(hook, id='secure_machine')


# 2. Make Fake data to send to each of them
### Schema; [age, sex, blood_type, weight, height, BMI]
    * Age: Integer
    * Sex: F = 0, M = 1
    * Blood Type: Should be one-hot encoded
    * Weight: Integer
    * Height: Inches (take ft*12 + in)
    * BMI: Float (assume the app calculates it!)

In [3]:
alex_tensor = torch.tensor([21, 1, 3, 165, 71, 23]).tag("#test")
toby_tensor = torch.tensor([19, 0, 1, 175, 73, 23.1]).tag("#test")
jake_tensor = torch.tensor([23, 1, 4, 123, 69, 18.2]).tag("#test")
matt_tensor = torch.tensor([28, 0, 7, 137, 82, 14.3]).tag("#test")
shreyas_tensor = torch.tensor([16, 1, 2, 167, 76, 20.3]).tag("#test")

In [4]:
# Send the tensor to the right worker node
alex_tensor = alex_tensor.send(alex)
toby_tensor = toby_tensor.send(toby)
jake_tensor = jake_tensor.send(jake)
matt_tensor = matt_tensor.send(matt)
shreyas_tensor = shreyas_tensor.send(shreyas)

### Way 1: Iterate Through Worker Array (not as scalable)

In [5]:
workers = [alex, toby, jake, matt, shreyas]

def calculate_means(workers):
    
    object1 = list(workers[0]._objects.values())
    mean = [0] * len(object1[0])
    
    for x in range(len(workers)):
        
        tensor = list(workers[x]._objects.values())
        
        for y in range(len(tensor[0])):
            mean[y] += (tensor[0][y].item() / len(workers))
            
    return mean

calculate_means(workers)

[21.4, 0.6000000000000001, 3.4, 153.4, 74.19999999999999, 19.78000011444092]

### Way 2: Iterate Through Workers (scalable!)

In [6]:
workers = [alex, toby, jake, matt, shreyas]
grid = sy.PrivateGridNetwork(*workers)

def calculate_means_with_grid(grid, tag):
    test_data = grid.search(tag)
    intermediate = list(test_data.values())
    
    mean = [0] * len(intermediate[0][0]) # set length of the averages you want to calculate
    
    for x in range(len(intermediate)):
        placeholder = intermediate[x][0].get()
        for y in range(len(placeholder)):
            val = placeholder[y].item()
            mean[y] += val / len(test_data)

    return mean
calculate_means_with_grid(grid, "#test")

[21.4, 0.6000000000000001, 3.4, 153.4, 74.19999999999999, 19.78000011444092]

### Different Step: Max & Min of each row

In [7]:
# Preprocessing of making tensors, sending, making a grid

# initiate tensors
alex_tensor = torch.tensor([21, 1, 3, 165, 71, 23]).tag("#test2")
toby_tensor = torch.tensor([19, 0, 1, 175, 73, 23.1]).tag("#test2")
jake_tensor = torch.tensor([23, 1, 4, 123, 69, 18.2]).tag("#test2")
matt_tensor = torch.tensor([28, 0, 7, 137, 82, 14.3]).tag("#test2")
shreyas_tensor = torch.tensor([16, 1, 2, 167, 76, 20.3]).tag("#test2")

# Send the tensor to the right worker node
alex_tensor = alex_tensor.send(alex)
toby_tensor = toby_tensor.send(toby)
jake_tensor = jake_tensor.send(jake)
matt_tensor = matt_tensor.send(matt)
shreyas_tensor = shreyas_tensor.send(shreyas)

workers = [alex, toby, jake, matt, shreyas]
grid = sy.PrivateGridNetwork(*workers)

In [8]:
def calculate_max_min_with_grid(grid, tag):
    
    test_data = grid.search(tag)
    intermediate = list(test_data.values())
    
    # instantiate a list of values to be filled
    max_list = [0] * len(intermediate[0][0]) 
    min_list = [1000000] * len(intermediate[0][0])
    
    for x in range(len(intermediate)):
        
        # this pulls the tensor
        placeholder = intermediate[x][0].get()
        for y in range(len(placeholder)):
            
            # gets the specific column value of the tensor
            val = placeholder[y].item()
            
            # update max or min
            if (val > max_list[y]):
                max_list[y] = val
            if (val < min_list[y]):
                min_list[y] = val

    return max_list, min_list

list_of_maxes, list_of_mins = calculate_max_min_with_grid(grid, "#test2")
print ("this is the list of maximum values per column:" + str(list_of_maxes))
print ("this is the list of minimum values per column:" + str(list_of_mins))

this is the list of maximum values per column:[28.0, 1, 7.0, 175.0, 82.0, 23.100000381469727]
this is the list of minimum values per column:[16.0, 0.0, 1.0, 123.0, 69.0, 14.300000190734863]
